# Homework 4

This Notebook builds on the DCOPF model introduced in [Notebook 6](https://github.com/east-winds/power-systems-optimization/tree/master/Notebooks) and incorporates some elements of Economic Dispatch introduced in [Notebook 4](https://github.com/east-winds/power-systems-optimization/tree/master/Notebooks).

First, load (or install if necessary) a set of packages you'll need for this assignment...

In [1]:
using JuMP
using GLPK
using DataFrames
using CSV
using Plots; plotly();

┌ Warning: For saving to png with the `Plotly` backend `PlotlyBase` and `PlotlyKaleido` need to be installed.
│   err = ArgumentError("Package PlotlyBase not found in current path.\n- Run `import Pkg; Pkg.add(\"PlotlyBase\")` to install the PlotlyBase package.")
└ @ Plots /Users/haydenburt/.julia/packages/Plots/4UTBj/src/backends.jl:425


## Question 1: Modifying IEEE-14

**A. Increased generation costs**

Copy the IEEE 14 bus system and DCOPF solver function from Notebook 6. In addition, add the following line to the return call of the function:
```julia
status = termination_status(DCOPF)
```
This tells you the solver termination status for the problem: e.g. was an optimal solution found, was the solution infeasible, was it unbounded, etc.

Make the following change to the system:

- Increase the variable cost of Generator 1 to \$30 / MWh

Run the DCOPF and output generation, flows, and prices.

In [21]:
# IEEE 14 Bus System Copied from Notebook 6

datadir = joinpath("..","Notebooks","ieee_test_cases") 
gens = CSV.read(joinpath(datadir,"Gen14.csv"), DataFrame);
lines = CSV.read(joinpath(datadir,"Tran14.csv"), DataFrame);
loads = CSV.read(joinpath(datadir,"Load14.csv"), DataFrame);

# Rename all columns to lowercase (by convention)
for f in [gens, lines, loads]
    rename!(f,lowercase.(names(f)))
end

# create generator ids 
gens.id = 1:nrow(gens);

# create line ids 
lines.id = 1:nrow(lines);
# add set of rows for reverse direction with same parameters
lines2 = copy(lines)
lines2.f = lines2.fromnode
lines2.fromnode = lines.tonode
lines2.tonode = lines2.f
lines2 = lines2[:,names(lines)]
append!(lines,lines2)

# calculate simple susceptance, ignoring resistance as earlier 
lines.b = 1 ./ lines.reactance

# keep only a single time period
loads = loads[:,["connnode","interval-1_load"]]
rename!(loads,"interval-1_load" => "demand");

In [22]:
# Solver function from Notebook 6 w/ return call modification to include status

#=
Function to solve DC OPF problem using IEEE test cases
Inputs:
    gen_info -- dataframe with generator info
    line_info -- dataframe with transmission lines info
    loads  -- dataframe with load info
=#
function dcopf_ieee(gens, lines, loads)
    DCOPF = Model(GLPK.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets based on data
      # Set of generator buses
    G = gens.connnode
    
      # Set of all nodes
    N = sort(union(unique(lines.fromnode), 
            unique(lines.tonode)))
    
      # sets J_i and G_i will be described using dataframe indexing below

    # Define per unit base units for the system 
    # used to convert from per unit values to standard unit
    # values (e.g. p.u. power flows to MW/MVA)
    baseMVA = 100 # base MVA is 100 MVA for this system
    
    # Decision variables   
    @variables(DCOPF, begin
        GEN[N]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        THETA[N]         # voltage phase angle of bus
        FLOW[N,N]        # flows between all pairs of nodes
    end)
    
    # Create slack bus with reference angle = 0; use bus 1 with generator
    fix(THETA[1],0)
                
    # Objective function
    @objective(DCOPF, Min, 
        sum( gens[g,:c1] * GEN[g] for g in G)
    )
    
    # Supply demand balances
    @constraint(DCOPF, cBalance[i in N], 
        sum(GEN[g] for g in gens[gens.connnode .== i,:connnode]) 
            + sum(load for load in loads[loads.connnode .== i,:demand]) 
        == sum(FLOW[i,j] for j in lines[lines.fromnode .== i,:tonode])
    )

    # Max generation constraint
    @constraint(DCOPF, cMaxGen[g in G],
                    GEN[g] <= gens[g,:pgmax])

    # Flow constraints on each branch; 
    # In DCOPF, line flow is a function of voltage angles
       # Create an array of references to the line constraints, 
       # which we "fill" below in loop
    cLineFlows = JuMP.Containers.DenseAxisArray{Any}(undef, 1:nrow(lines)) 
    for l in 1:nrow(lines)
        cLineFlows[l] = @constraint(DCOPF, 
            FLOW[lines[l,:fromnode],lines[l,:tonode]] == 
            baseMVA * lines[l,:b] * 
            (THETA[lines[l,:fromnode]] - THETA[lines[l,:tonode]])
        )
    end
    
    # Max line flow limits
       # Create an array of references to the line constraints, 
       # which we "fill" below in loop
    cLineLimits = JuMP.Containers.DenseAxisArray{Any}(undef, 1:nrow(lines)) 
    for l in 1:nrow(lines)
        cLineLimits[l] = @constraint(DCOPF,
            FLOW[lines[l,:fromnode],lines[l,:tonode]] <=
            lines[l,:capacity]
        ) 
    end

    # Solve statement (! indicates runs in place)
    optimize!(DCOPF)

    # Output variables
    generation = DataFrame(
        node = gens.connnode,
        gen = value.(GEN).data[gens.connnode]
        )
    
    angles = value.(THETA).data
    
    flows = DataFrame(
        fbus = lines.fromnode,
        tbus = lines.tonode,
        flow = baseMVA * lines.b .* (angles[lines.fromnode] .- 
                        angles[lines.tonode]))
    test = value.(GEN)
    
    # We output the marginal values of the demand constraints, 
    # which will in fact be the prices to deliver power at a given bus.
    prices = DataFrame(
        node = N,
        value = dual.(cBalance).data)
    
    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        angles,
        flows,
        prices,
        cost = objective_value(DCOPF),
        status = termination_status(DCOPF)
    )
end

dcopf_ieee (generic function with 1 method)

In [23]:
# Modify Generator 1 variable cost to $30/MWh

gens_newcost = copy(gens)
gens_newcost[1,:c1] = 30
gens_newcost

Row,connnode,c2,c1,c0,pgmax,pgmin,rgmax,rgmin,pgprev,id
,Int64,Float64,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64
1,1,0.0430293,30,0,332.4,0,100,-100,161.619,1
2,2,0.25,20,0,140.0,0,100,-100,97.4704,2


In [24]:
solution = dcopf_ieee(gens_newcost, lines, loads);

In [25]:
solution.generation

Row,node,gen
,Int64,Float64
1,1,119.0
2,2,140.0


In [26]:
solution.flows

Row,fbus,tbus,flow
,Int64,Int64,Float64
1,1,2,64.0779
2,1,5,54.9221
3,2,3,72.7846
4,2,4,60.9488
5,2,5,48.6446
6,3,4,-21.4154
7,4,5,-54.3377
8,4,7,29.274
9,4,9,16.7971


In [27]:
solution.prices

Row,node,value
,Int64,Float64
1,1,30.0
2,2,30.0
3,3,30.0
4,4,30.0
5,5,30.0
6,6,30.0
7,7,30.0
8,8,30.0
9,9,30.0


Regarding the above results, answer the following:

- How has generation changed compared to the default system?
- What explains the new prices?


Generation has not changed - in both cases, Generator 1 generates 119 MW and Generator 2 generates 140 MW. However, in this case, the prices have changed. Previously, the price at every location was \\$20/MWh but is now \\$30/MWh. This suggests that 

**B. Constrained line**

Make the following changes to the system:

- Increase the variable cost of Generator 1 to \$30 / MWh
- Reduce flow limit on the line connecting 2 and 3 ($l_{23}$) to 70 MW

Run the DCOPF and output generation, flows, and prices.

In [28]:
# Reduce the flow limit on l_23 to 70 MW

lines_constrained = copy(lines)
lines_constrained[3,:capacity] = 70
lines_constrained

Row,fromnode,tonode,resistance,reactance,contingencymarked,capacity,id,b
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64
1,1,2,0.01938,0.05917,1,10000,1,16.9005
2,1,5,0.05403,0.22304,1,10000,2,4.4835
3,2,3,0.04699,0.19797,1,70,3,5.05127
4,2,4,0.05811,0.17632,1,10000,4,5.67151
5,2,5,0.05695,0.17388,1,10000,5,5.75109
6,3,4,0.06701,0.17103,1,10000,6,5.84693
7,4,5,0.01335,0.04211,1,10000,7,23.7473
8,4,7,0.0,0.20912,1,10000,8,4.78194
9,4,9,0.0,0.55618,1,10000,9,1.79798


In [29]:
solution_constrained_line = dcopf_ieee(gens_newcost, lines_constrained, loads);

In [30]:
solution_constrained_line.generation

Row,node,gen
,Int64,Float64
1,1,220.837
2,2,38.1627


In [31]:
solution_constrained_line.flows

Row,fbus,tbus,flow
,Int64,Int64,Float64
1,1,2,149.42
2,1,5,71.417
3,2,3,70.0
4,2,4,55.1212
5,2,5,40.7618
6,3,4,-24.2
7,4,5,-62.4868
8,4,7,28.9798
9,4,9,16.6283


In [32]:
solution_constrained_line.prices

Row,node,value
,Int64,Float64
1,1,30.0
2,2,20.0
3,3,224.574
4,4,85.3586
5,5,67.6948
6,6,73.7014
7,7,82.1916
8,8,82.1916
9,9,80.5255


Regarding the above results, answer the following:

- Which node has the highest price and why?
- What is the difference in prices across $l_{23}$, also known as the congestion rent? How do you interpret this value (what is it's practical meaning?)

Node 3 has the highest price at \$224.57/MWh. 

**C. Demand increase**

Make the following changes to the system:

- Increase the variable cost of Generator 1 to \$30 / MWh
- Reduce flow limit on the line connecting 2 and 3 ($l_{23}$) to 70 MW
- Increase demands everywhere by 5\%.

In [33]:
loads_inc = copy(loads)
loads_inc[!,:demand] = loads_inc[!,:demand].*1.05
loads_inc

Row,connnode,demand
,Int64,Float64
1,2,-22.785
2,3,-98.91
3,4,-50.19
4,5,-7.98
5,6,-11.76
6,9,-30.975
7,10,-9.45
8,11,-3.675
9,12,-6.405


Calculate the total available generating capacity:

In [34]:
total_gen_cap = sum(gens_newcost[!,:pgmax])

472.4

Calculate the new total demand:

In [35]:
total_demand = sum(loads_inc[!,:demand])

-271.94999999999993

Run the DCOPF and show prices.

In [36]:
solution_inc_demand = dcopf_ieee(gens_newcost, lines_constrained, loads_inc);

In [37]:
solution_inc_demand.prices

Row,node,value
,Int64,Float64
1,1,5.32907e-15
2,2,-1.0
3,3,19.4574
4,4,5.53586
5,5,3.76948
6,6,4.37014
7,7,5.21916
8,8,5.21916
9,9,5.05255


In [38]:
solution_inc_demand.flows

Row,fbus,tbus,flow
,Int64,Int64,Float64
1,1,2,264.16
2,1,5,95.7207
3,2,3,70.0
4,2,4,50.5525
5,2,5,32.8917
6,3,4,-28.91
7,4,5,-75.8539
8,4,7,30.059
9,4,9,17.2475


In [39]:
solution_inc_demand.generation

Row,node,gen
,Int64,Float64
1,1,359.881
2,2,-87.9306


In [40]:
solution_inc_demand.status

INFEASIBLE::TerminationStatusCode = 2

**What is happening in this system?** 

This system results in an infeasible solution

## Question 2: Linear losses

Up until now, we have ignored transmission losses. A quadratic approximation of losses is given by:

\begin{align}
LOSS_{ij} &\approx \frac{G_{ij}}{BaseMVA} (\theta_i-\theta_j)^2 \\
 & \approx \frac{1}{BaseMVA} \frac{R_{ij}}{R_{ij}^2+X_{ij}^2}(\theta_i-\theta_j)^2
\end{align}


where $G$ is the line's conductance, $R$ is the line's resistance and $X$ is the line's reactance. See the `lines` data frame for these parameters.

For our purposes, we will approximate this quadratic via:


$$
LOSS_{ij} \geq \frac{R_{ij}}{BaseMVA} \times (MaxFlow_{ij})^2 
\left(\frac{|FLOW_{ij}|}{MaxFlow_{ij}} - 0.165 \right)
$$

where $MaxFlow_{ij}=200 MW$ in this problem. Note the greater than equal sign, as we do not want to have negative losses.

This approximation is based on Fitiwi et al. (2016), "Finding a representative network losses model for large-scale transmission expansion planning with renewable energy sources," *Energy* 101: 343-358, https://doi.org/10.1016/j.energy.2016.02.015. 

Note that this is a linear approximation of transmission losses, which are actually a quadratic function of power flows. Fitiwi et al. 2016 and other papers describe piece-wise or segment-wise linear approximations of the quadratic function which provide a tighter lower bound approximation of losses, but we'll use a single linear term for this assignment. 

See Jenkins & Sepulveda et al. 2017, "Enhanced decision support for a changing electricity landscape: the GenX configurable electricity resource capacity expansion model", MIT Energy Initiative Working Paper 2017-10 http://bit.ly/GenXModel Section 5.8, for an example of a linear segment-wise approximation of quadratic transmission losses. 


**A. Code linear losses**

Reload the original data from Notebook 6 and copy the IEEE 14 bus system and DCOPF solver function from Notebook 6 into a new function `dcopf_ieee_lossy`.

Make the following changes:
- Increase the variable cost of Generator 1 to \$30 / MWh
- Change all transmission line capacities to 200 MW

Implement losses into the supply/demand balance equations. A standard way to implement absolute values in linear programming is by introducing two non-negative auxiliary variables $x^+$, $x^-$ $\geq 0$:

$$
x = x^+ - x^-
$$

and the absolute value can be represented as:

$$
|x| = x^+ + x^-
$$

(You should satisfy yourself that this equality holds.)

It makes the formulation easier if losses are added to the supply/demand balance constraint in each node by splitting losses in half between the receiving and sending end.

Indicate which equations and variables you have added and explain your steps using inline code comments (e.g. `# Comment`).

Run the lossy DCOPF and output generation, flows, losses, and prices.

In [48]:
gens_losses = copy(gens)
gens_losses[1,:c1] = 30

30

In [55]:
lines_losses = copy(lines)
lines_losses[!,:capacity] .= 200;

In [99]:
#=
Function to solve DC OPF problem using IEEE test cases
Inputs:
    gen_info -- dataframe with generator info
    line_info -- dataframe with transmission lines info
    loads  -- dataframe with load info
=#
function dcopf_ieee_lossy(gens, lines, loads)
    DCOPF = Model(GLPK.Optimizer) 
    
    # Define sets based on data
      # Set of generator buses
    G = gens.connnode
    
      # Set of all nodes
    N = sort(union(unique(lines.fromnode), 
            unique(lines.tonode)))
    
      # sets J_i and G_i will be described using dataframe indexing below

    # Define per unit base units for the system 
    # used to convert from per unit values to standard unit
    # values (e.g. p.u. power flows to MW/MVA)
    baseMVA = 100 # base MVA is 100 MVA for this system
    
    # Decision variables   
    @variables(DCOPF, begin
        GEN[N]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        THETA[N]         # voltage phase angle of bus
        FLOW[N,N]       # flows between all pairs of nodes
        FLOW_PLUS[N,N] >= 0        # auxiliary variable for flow
        FLOW_MINUS[N,N] >= 0       # auxiliary variable for flow
        LOSS[N,N] # losses in lines
    end)
    
    # Create slack bus with reference angle = 0; use bus 1 with generator
    fix(THETA[1],0)
                
    # Objective function
    @objective(DCOPF, Min, 
        sum( gens[g,:c1] * GEN[g] for g in G)
    )
    
    # Flow auxiliary variables definition constraint
    @constraint(DCOPF, cAux[l in 1:nrow(lines)], 
        FLOW[lines[l,:fromnode],lines[l,:tonode]] == FLOW_PLUS[lines[l,:fromnode],lines[l,:tonode]]-FLOW_MINUS[lines[l,:fromnode],lines[l,:tonode]]
    )
    
    # Losses constraint
    cLosses = JuMP.Containers.DenseAxisArray{Any}(undef, 1:nrow(lines))
    for l in 1:nrow(lines)
        cLosses[l] = @constraint(DCOPF,
            LOSS[lines[l,:fromnode],lines[l,:tonode]] >= 
            (lines[l,:resistance]/baseMVA)*(lines[l,:capacity]*lines[l,:capacity])*(((FLOW_PLUS[lines[l,:fromnode],lines[l,:tonode]]+FLOW_MINUS[lines[l,:fromnode],lines[l,:tonode]])./lines[l,:capacity])-0.165)
        )
    end
    
    # Supply demand balances
    @constraint(DCOPF, cBalance[i in N], 
        sum(GEN[g] for g in gens[gens.connnode .== i,:connnode]) 
            + sum(load for load in loads[loads.connnode .== i,:demand]) 
        == sum(FLOW[i,j]+LOSS[i,j] for j in lines[lines.fromnode .== i,:tonode]) # losses are now included 
    )

    # Max generation constraint
    @constraint(DCOPF, cMaxGen[g in G],
                    GEN[g] <= gens[g,:pgmax])

    # Flow constraints on each branch; 
    # In DCOPF, line flow is a function of voltage angles
       # Create an array of references to the line constraints, 
       # which we "fill" below in loop
    cLineFlows = JuMP.Containers.DenseAxisArray{Any}(undef, 1:nrow(lines)) 
    for l in 1:nrow(lines)
        cLineFlows[l] = @constraint(DCOPF, 
            FLOW[lines[l,:fromnode],lines[l,:tonode]] == 
            baseMVA * lines[l,:b] * 
            (THETA[lines[l,:fromnode]] - THETA[lines[l,:tonode]])
        )
    end
    
    # Max line flow limits
       # Create an array of references to the line constraints, 
       # which we "fill" below in loop
    cLineLimits = JuMP.Containers.DenseAxisArray{Any}(undef, 1:nrow(lines)) 
    for l in 1:nrow(lines)
        cLineLimits[l] = @constraint(DCOPF,
            FLOW[lines[l,:fromnode],lines[l,:tonode]] <=
            lines[l,:capacity]
        ) 
    end

    # Solve statement (! indicates runs in place)
    optimize!(DCOPF)

    # Output variables
    generation = DataFrame(
        node = gens.connnode,
        gen = value.(GEN).data[gens.connnode]
        )
    
    angles = value.(THETA).data
    
    flows = DataFrame(
        fbus = lines.fromnode,
        tbus = lines.tonode,
        flow = baseMVA * lines.b .* (angles[lines.fromnode] .- 
                        angles[lines.tonode]))
    
    # We output the marginal values of the demand constraints, 
    # which will in fact be the prices to deliver power at a given bus.
    prices = DataFrame(
        node = N,
        value = dual.(cBalance).data)
    
    # provide the losses as output
    losses = value.(LOSS).data
    
    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        angles,
        flows,
        prices,
        losses,
        cost = objective_value(DCOPF),
        status = termination_status(DCOPF)
    )
end

dcopf_ieee_lossy (generic function with 1 method)

In [100]:
solution_losses = dcopf_ieee_lossy(gens_losses, lines_losses, loads);

In [101]:
solution_losses.generation

Row,node,gen
,Int64,Float64
1,1,29.6021
2,2,140.0


In [102]:
solution_losses.flows

Row,fbus,tbus,flow
,Int64,Int64,Float64
1,1,2,9.17622
2,1,5,22.4855
3,2,3,62.8772
4,2,4,37.3214
5,2,5,25.7201
6,3,4,-34.3056
7,4,5,-50.0663
8,4,7,2.63645
9,4,9,1.51277


In [103]:
solution_losses.losses

14×14 Matrix{Float64}:
  0.0      -0.92341   0.0       0.0       …    0.0        0.0       0.0
 -0.92341   0.0       2.80786   0.502232       0.0        0.0       0.0
  0.0       2.80786   0.0       0.174981       0.0        0.0       0.0
  0.0       0.502232  0.174981  0.0            0.0        0.0       0.0
 -1.13619  -0.829178  0.0       0.455671       0.0        0.0       0.0
  0.0       0.0       0.0       0.0       …   -6.34427   -3.57232   0.0
  0.0       0.0       0.0       0.0            0.0        0.0       0.0
  0.0       0.0       0.0       0.0            0.0        0.0       0.0
  0.0       0.0       0.0       0.0            0.0        0.0      -6.14073
  0.0       0.0       0.0       0.0            0.0        0.0       0.0
  0.0       0.0       0.0       0.0       …    0.0        0.0       0.0
  0.0       0.0       0.0       0.0            0.0      -12.2415    0.0
  0.0       0.0       0.0       0.0          -12.2415     0.0      -7.99715
  0.0       0.0       0.0       0

In [112]:
solution_losses.prices

Row,node,value
,Int64,Float64
1,1,30.0
2,2,32.5304
3,3,45.8675
4,4,39.5736
5,5,36.8181
6,6,26.8452
7,7,44.8318
8,8,44.8318
9,9,47.598


**B. Interpret results**

Run the same parameters in the lossless OPF from problem 1. How do prices and flows change? What is the largest magnitude difference in prices between the solution with losses and the lossless OPF solution?

In [107]:
solution_no_losses = dcopf_ieee(gens_losses, lines_losses, loads);

In [108]:
solution_no_losses.generation

Row,node,gen
,Int64,Float64
1,1,119.0
2,2,140.0


In [109]:
solution_no_losses.flows

Row,fbus,tbus,flow
,Int64,Int64,Float64
1,1,2,64.0779
2,1,5,54.9221
3,2,3,72.7846
4,2,4,60.9488
5,2,5,48.6446
6,3,4,-21.4154
7,4,5,-54.3377
8,4,7,29.274
9,4,9,16.7971


In [110]:
solution_no_losses.prices

Row,node,value
,Int64,Float64
1,1,30.0
2,2,30.0
3,3,30.0
4,4,30.0
5,5,30.0
6,6,30.0
7,7,30.0
8,8,30.0
9,9,30.0


In [113]:
show(solution_losses.prices[!,:value]-solution_no_losses.prices[!,:value])

[-3.552713678800501e-14, 2.5303969499254038, 15.867500733381227, 9.573642582083977, 6.818088649773731, -3.1548362317783827, 14.831836379097744, 14.831836379097712, 17.59797017600613, 13.491842434757782, 4.527937482246124, -18.589977508158523, -8.968680650349473, 2.524326160025872]

The prices and flows change ...

The largest magnitude difference is ...

## Question 3 - Security contingencies

Power system operators need to ensure that power is delivered reliably even in the event of unexpected outages (**contingencies**). One common contigency that must be planned for is the loss of a transmission line. The security-constrained OPF (SCOPF) run by operators solves for an optimal dispatch that is simultaneously robust (i.e., feasible) to each of the lines failing individually. This is what is known as **N-1 security**, because we assume that at most one component fails in any given scenario.

In this problem, we will not code a full SCOPF, but rather investigate what happens to the feasibility of our problem when we remove transmission lines.

**A. Setup data**

The following code loads the original dataset (with one row per line) and includes a function `format_lines` that converts this to a format that our solver function can use (duplicating rows for both directions, adding susceptance, etc.).

In [114]:
lines = CSV.read(joinpath(datadir,"Tran14.csv"), DataFrame);
rename!(lines,lowercase.(names(lines)))

function format_lines(lines)
    # create line ids 
    lines.id = 1:nrow(lines);
    # add set of rows for reverse direction with same parameters
    lines2 = copy(lines)
    lines2.f = lines2.fromnode
    lines2.fromnode = lines.tonode
    lines2.tonode = lines2.f
    lines2 = lines2[:,names(lines)]
    append!(lines,lines2)

    # calculate simple susceptance, ignoring resistance as earlier 
    lines.b = 1 ./ lines.reactance
    return(lines)
end

format_lines (generic function with 1 method)

Next:

1. Set the capacity of all lines in the system at 100 MW, except for the line $l_{12}$, which you should set to 200 MW.

2. Create a load dataframe `loads_sens` that increases demands everywhere by 10\%

In [123]:
lines_orig = format_lines(lines)
lines_new_cap = copy(lines_orig)
lines_new_cap[!,:capacity] .= 100
lines_new_cap[1,:capacity] = 200;

In [128]:
loads_sens = copy(loads)
loads_sens[!,:demand] = loads_sens[!,:demand].*1.10;

**B. Loop over line contingencies**

Create a dataframe `status` with the `fromnode` and `tonode` columns of `lines`.

Create a [for loop](https://docs.julialang.org/en/v1/manual/control-flow/#man-loops) that iterates over each line and:
- sets the reactance to be a very high value, 1e9 (i.e., no power will be transmitted)
- creates a version of the dataframe that our solver function can use via `format_lines`
- runs DCOPF
- stores the solution status in a `opf` column in the corresponding row of the `status` dataframe

Show the `status` results.

In [154]:
status = DataFrame(
    fromnode = lines_new_cap[!,:fromnode],
    tonode = lines_new_cap[!,:tonode],
    opf = solution.status
);

In [155]:
for i in 1:nrow(lines_new_cap)
    lines_temp = copy(lines_new_cap)
    lines_temp[i,:reactance] = 10^9
    lines_temp = format_lines(lines_temp)
    sol = dcopf_ieee(gens, lines_temp, loads_sens)
    status[i,:opf] = sol.status
end


In [172]:
num_feasible = 0
num_infeasible = 0
for i in 1:nrow(lines_new_cap)
    if status[i,:opf] == OPTIMAL
        num_feasible = num_feasible + 1
    else status[i,:opf] == INFEASIBLE
        num_infeasible = num_infeasible + 1
    end
end              
print("Feasible: ", num_feasible, ", Infeasible: ", num_infeasible)

Feasible: 256, Infeasible: 4864

**3. Interpret results**

Are all of the cases feasible? If not, how many are infeasible? 

Pick two cases where the solution gives a different status. (For our purposes, dual infeasible and primal infeasible are the same.) What is happening here?

Given this, do you conclude that the system with the assumed transmission line ratings is secure as-is, or do we need to add more redundancy to the system?